## Imports

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
%cd ..
import os, sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

/Users/Tony/Other Docs/distilling-and-forgetting-in-large-pre-trained-models


In [38]:
from functools import partial

from transformers.models.whisper import (WhisperTokenizer,
                                         WhisperTokenizerFast,
                                         WhisperFeatureExtractor,
                                         WhisperForConditionalGeneration)

from dataloader.dataset_for_training.dataset_loader_ami import load_ami_100h
from dataloader.preprocessing_train.preprocessing import prepare_dataset_fct
from evaluation.eval_dataset_name_to_dataset_group import EVAL_DATASET_NAME_TO_DATASET_GROUP
from utils.whisper_hallucinations.get_features import get_audio_length_in_seconds

if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():  # for Apple Silicon
    device = torch.device("mps")
else:
    device = "cpu"

## Load the model

In [22]:
pretrained_model_name_or_path = "openai/whisper-tiny"

model = WhisperForConditionalGeneration.from_pretrained(pretrained_model_name_or_path)
feature_extractor = WhisperFeatureExtractor.from_pretrained(pretrained_model_name_or_path)
tokenizer = WhisperTokenizerFast.from_pretrained(pretrained_model_name_or_path, language="english", task="transcribe")

model.generate = partial(model.generate, task="transcribe", max_length=255, use_cache=True)

## Load test dataset

In [32]:
dataset_name = "ami"

ds_test = EVAL_DATASET_NAME_TO_DATASET_GROUP[dataset_name]()[dataset_name]

# if dataset_name == "ami":
#     ds = ds.select(list(range(32)))

Found cached dataset ami (/Users/Tony/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5)


In [33]:
prepare_dataset = partial(prepare_dataset_fct, tokenizer=tokenizer, feature_extractor=feature_extractor)
ds_test = ds_test.map(prepare_dataset, num_proc=1).with_format("pt")

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [34]:
ds_test = ds_test.map(get_audio_length_in_seconds, num_proc=1)

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [37]:
ds_test["audio_length"].sum() / 3600

tensor(8.6815)

In [36]:
ds_test.num_rows

12643

## Load validation dataset

In [41]:
ds_val = load_ami_100h()["validation"]

Found cached dataset ami (/Users/Tony/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5)
Found cached dataset ami (/Users/Tony/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5)


In [42]:
prepare_dataset = partial(prepare_dataset_fct, tokenizer=tokenizer, feature_extractor=feature_extractor)
ds_val = ds.map(prepare_dataset, num_proc=1).with_format("pt")

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [43]:
ds_val = ds_val.map(get_audio_length_in_seconds, num_proc=1)

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [44]:
ds_val["audio_length"].sum() / 3600

tensor(8.6815)

In [45]:
ds_val.num_rows

12643